# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링

### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?

In [46]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

import pandas as pd
from selenium.webdriver.common.by import By

https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2
https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2
https://m.place.naver.com/my/5bd2f3682db9d1bdd38eedca/review?v=2
https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2
https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2

In [47]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# 주소 이동 (TEST)
url = 'https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2'
driver.get(url)
time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

### User 정보 찾기

In [3]:
# User ID 찾기
user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
user_id = user_id_element.text if user_id_element else None

# 리뷰, 팔로잉, 팔로워 찾기 
user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
user_element = {}
for element in user_popularity:
    em_tag = element.find('em')
    if em_tag:
        key = element.text.replace(em_tag.text, '').strip()
        value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
        user_element[key] = value

print('아이디 :', user_id,user_element)

아이디 : JUDY0725 {'리뷰': 1921, '팔로잉': 0, '팔로워': 95}


### 매장 정보 찾기

In [54]:
# 맨 처음 게시물 
button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
button.click()
time.sleep(0.4)
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [48]:
soup

<html><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no, user-scalable=no" name="viewport"/><title>JUDY0725님의 MY플레이스</title><meta content="JUDY0725님의 MY플레이스" name="title"/><meta content="나만의 장소 리스트를 만들고, 영수증으로 진짜 리뷰도 남겨보세요!" name="description"/><meta content="https://m.place.naver.com/my/5bf92274b7236e3778d7c30d/review?v=2" property="og:url"/><meta content="JUDY0725님의 MY플레이스" property="og:title"/><meta content="나만의 장소 리스트를 만들고, 영수증으로 진짜 리뷰도 남겨보세요!" property="og:description"/><meta content="https://g-myplace.pstatic.net/cdn/240206-144324/4.8.10/public/web/static/images/og_tag_default.png" property="og:image"/><meta content="1200" property="og:image:width"/><meta content="630" property="og:image:height"/><meta content="네이버 MY플레이스" property="article:author"/><meta content="네이버 MY플레이스" property="article:publisher"/><meta content="summary_large_image" property="twitter:card"/><meta content="https://m.place.naver.com/my/5bf92274b7236e3778d7c30d

In [51]:
# 클래스명이 'ptr__children'인 모든 요소를 찾음
ptr_children_elements = driver.find_elements(By.CLASS_NAME, 'ptr__children')

# 각 요소의 텍스트를 출력
for element in ptr_children_elements:
    print(element.text)


네이버
MY플레이스
리뷰 참여
프로필
JUDY0725
리뷰1,921팔로잉0팔로워95
Hello! I'm Judy from Korea :)
팔로우
리뷰어 관리
최신순조회수순
사진/영상 리뷰만
전체709
베이커리36
리뷰 많이 남긴 메뉴
를 먼저 보여드려요
툴팁 닫기
칼국수,만두11
브런치9
카페,디저트264
한식106
양식45
쇼핑,유통
파스텔드나따 압구정 리뷰
서울시 신사동
파스텔드나따 압구정
단데농 리뷰
서울시 창천동
단데농
스윗센츠 리뷰
서울시 양재동
스윗센츠
덴키카츠 리뷰
서울시 삼성동
덴키카츠
우드왁싱 선릉점 리뷰
서울시 대치동
우드왁싱 선릉점
야익스커피 리뷰
대전시 월평동
야익스커피
태희갤러리카페 리뷰
대전시 장대동
태희갤러리카페
창억떡집 대전점 리뷰
대전시 장대동
창억떡집 대전점
따봉하우스 리뷰
서울시 이문동
따봉하우스
보자기두부 서초점 리뷰
서울시 서초동
보자기두부 서초점
써니드플레르 교대역점 리뷰
서울시 서초동
써니드플레르 교대역점
소울버킷 리뷰
서울시 서초동
소울버킷
더소뇨 강남점 리뷰
서울시 신사동
더소뇨 강남점
뷰티엘라 강남 리뷰
서울시 역삼동
뷰티엘라 강남
에그드랍 서초역점 리뷰
서울시 서초동
에그드랍 서초역점
심재 선릉 리뷰
서울시 대치동
심재 선릉
Bong 커피 리뷰
서울시 논현동
Bong 커피
MUJI 파르나스몰점 리뷰
서울시 삼성동
MUJI 파르나스몰점
로딩중
로그인전체서비스
이용정책MY플레이스 고객센터신고센터공지사항
© NAVER Corp.


### 리뷰 크롤링

- 아직 안된 부분 : pagedown하기 , tag 부분


In [91]:
# data를 담을 빈 리스트 선언
data_list = [] 

# 리뷰 페이지를 크롤링
ptr_children_elements = driver.find_elements(By.CLASS_NAME, 'ptr__children')

# 페이지에서 각각 클래스 찾기 
for ptr_element in ptr_children_elements:
    restaurant_elements = ptr_element.find_elements(By.CLASS_NAME, '_1QGRWW') 
    category_location_elements = ptr_element.find_elements(By.CLASS_NAME, '_2vBfgu')
    review_text_elements = ptr_element.find_elements(By.CLASS_NAME, 'z0t_8b')
    reactions_elements = ptr_element.find_elements(By.CLASS_NAME, 'COw42b')
    reactions = [span.text for span in driver.find_element(By.CLASS_NAME, '_1oWLID').find_elements(By.CLASS_NAME, 'COw42b')]
    date_info = driver.find_element(By.CLASS_NAME, '_15xwjO').find_element(By.CLASS_NAME, 'hol3Ic').find_element(By.TAG_NAME, 'div')

    # 레스토랑 클래스가 존재하는 경우, 모든 정보를 가져와서 딕셔너리로 저장하기
    for i in range(len(restaurant_elements)) :
        if restaurant_elements:
            restaurant = restaurant_elements[i].text if len(restaurant_elements[i].text) > 0 else '없음'
            category_location = category_location_elements[i].text if len(category_location_elements[i].text) > 0 else '없음'
            # category = category_location[0].text if len(category_location) > 0 else '없음'
            # location = category_location[1].text if len(category_location) > 1 else '없음'
            review_text = review_text_elements[i].text if len(review_text_elements[i].text) > 0 else '없음'
            reactions = reactions if len(reactions) > 0 else '없음'
            date_element = date_info.find_element(By.XPATH, './span[@class="_3nNYBi"]/time') # 여기 수정
            day = date_element.text if date_element else '없음'
            
            # 딕셔너리 만들고, list로 추가하기
            data_dict = {
                '매장명': restaurant,
                '매장정보': category_location,
                '리뷰 내용': review_text,
                '태그': reactions,
                '방문일자': day
            }
            data_list.append(data_dict)
    
# 결과 확인
df = pd.DataFrame(data_list)
df

,매장명,매장정보,리뷰 내용,태그,방문일자
0,파스텔드나따 압구정,베이커리서울특별시 강남구 신사동,일요일 오후에 방문했는데 직원분 너무 친절하고 겸손하셔서 놀라고 갑니다! 에그타르트...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
1,단데농,"카페,디저트서울특별시 서대문구 창천동",결혼식 가는길 버스 대기시간이 길어서 들렸는데 사장님 너무 예쁘시고 런더포그 너무 ...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
2,스윗센츠,"카페,디저트서울특별시 서초구 양재동",양재청 로스터리 카페 스윗센츠! 직접 로스팅을 해서 커피맛이 신선하구요! 시그니처 ...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
3,덴키카츠,돈가스서울특별시 강남구 삼성동,돈가스도 카레우동도 다\n맛있어요!!!,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
4,우드왁싱 선릉점,"왁싱,제모서울특별시 강남구 대치동",샵이 깨끗하고 쾌적해요!\n시술도 정말 꼼꼼하게 잘 해주시고 마무리 케어도 훌륭해요...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
5,야익스커피,카페대전광역시 서구 월평동,커피맛 깔끔하고 파운드 케이크도 너무 맛있어요!\n분위기가 조용해서 작업하기도 좋고...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
6,태희갤러리카페,"카페,디저트대전광역시 유성구 장대동",조용하고 좋아요!,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
7,창억떡집 대전점,"떡,한과대전광역시 유성구 장대동",대전 놀러왔다가 창억떡집 보고 신기해서 들어갔다가 밤통팥찰떡 샀어요! 찰떡 사면 재...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
8,따봉하우스,떡볶이서울특별시 동대문구 이문동,친구가 20년 다닌 맛집이라며 데려가줬는데 예스러움에 반하고 맛있고 친절+유쾌하셔서...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일
9,보자기두부 서초점,두부제조서울특별시 서초구 서초동,이름도 너무 예쁜 보자기두부! 매장에서 직접 만드는 두부라 안심하고 먹을 수 있어요...,"[특별한 메뉴가 있어요, 디저트가 맛있어요, 친절해요]",2.4.일


### 매장 리뷰 크롤링

In [41]:
# 매장명
restaurant = driver.find_element(By.CLASS_NAME, '_1QGRWW').text
restaurant = restaurant if len(restaurant) > 0 else '없음'

# 주소 
category_location = driver.find_element(By.CLASS_NAME, '_2vBfgu').find_elements(By.TAG_NAME, 'span')
category = category_location[0].text if len(category_location) > 0 else '없음'
location = category_location[1].text if len(category_location) > 1 else '없음'

# 리뷰
review_text = driver.find_element(By.CLASS_NAME, 'z0t_8b').find_element(By.CLASS_NAME, '_3-ITu7').text
review_text = review_text if len(review_text) > 0 else '없음'

# tag 
reactions = [span.text for span in driver.find_element(By.CLASS_NAME, '_1oWLID').find_elements(By.CLASS_NAME, 'COw42b')]
reactions = reactions if len(reactions) > 0 else '없음'

# 방문일자 
date_info = driver.find_element(By.CLASS_NAME, '_15xwjO').find_element(By.CLASS_NAME, 'hol3Ic').find_element(By.TAG_NAME, 'div')
date_element = date_info.find_element(By.XPATH, './span[@class="_3nNYBi"]/time')
day = date_element.text if date_element else '없음'

# Create a DataFrame
data = {
    # '아이디' : [user_id],
    # '리뷰' : user_element['리뷰'],
    # '팔로워' : user_element['팔로워'],
    '매장명': [restaurant],
    '카테고리': [category],
    '주소': [location],
    '리뷰 내용': [review_text],
    '태그': [reactions],
    '방문일자': [day]
}

User_df = pd.DataFrame(data)